3.1

For an input of size ci x h x w, a kernel of co x ci x kh x kw, padding (ph,pw) and stride(sh,sw) we can break it down into costs per single value in each of the output channels, then costs per output channel, then total costs.

Each output value is the conclusion of multiplying kh x kw input numbers in each input channel and then adding them all together. That is, they require kh x kw multiplications per input channel, meaning kh x kw x ci total multiplications per single output value. The number of additions is thus that number minus one, kh x kw x ci - 1.

Each output channel has a size of floor((h+ph-kh+1)/sh,(w+pw-kw+1)/sw), meaning the total number of values in an output channel is floor((h+ph-kh+1)/sh) x floor((w+pw-kw+1)/sw). If there are co output channels this results in a total of co x floor((h+ph-kh+1)/sh) x floor((w+pw-kw+1)/sw) values to be outputted.

Thus, the total computational costs are:


Multiplication: kh x kw x ci x co x floor((h+ph-kh+1)/sh) x floor((w+pw-kw+1)/sw)

Addition: (kh x kw x ci - 1) x co x floor((h+ph-kh+1)/sh) x floor((w+pw-kw+1)/sw)

3.2

The memory footprint of forward propogation is the size of the kernel times 2 ie co x ci x kh x kw x 2

3.3

The memory footprint of the forward propogation is the size of the kernel times 3 (kernel value, gradient and un-activated solution)

3.4

The computational cost of the backward propogation is equivalent to the twice the amount of the size of the kernel as each point in the kernel needs to have the gradient calculated, and then has to be altered by an appropriate amount in respect to the gradient.

In [ ]:
# note this is just to check how many max comparisons are needed to reach an exact solution
import math
for i in range(100):
  count = 0
  num = i
  while num > 1:
    num = num - math.floor(num/2)
    count = count + 1
  print(i, count)

0 0
1 0
2 1
3 2
4 2
5 3
6 3
7 3
8 3
9 4
10 4
11 4
12 4
13 4
14 4
15 4
16 4
17 5
18 5
19 5
20 5
21 5
22 5
23 5
24 5
25 5
26 5
27 5
28 5
29 5
30 5
31 5
32 5
33 6
34 6
35 6
36 6
37 6
38 6
39 6
40 6
41 6
42 6
43 6
44 6
45 6
46 6
47 6
48 6
49 6
50 6
51 6
52 6
53 6
54 6
55 6
56 6
57 6
58 6
59 6
60 6
61 6
62 6
63 6
64 6
65 7
66 7
67 7
68 7
69 7
70 7
71 7
72 7
73 7
74 7
75 7
76 7
77 7
78 7
79 7
80 7
81 7
82 7
83 7
84 7
85 7
86 7
87 7
88 7
89 7
90 7
91 7
92 7
93 7
94 7
95 7
96 7
97 7
98 7
99 7


In [ ]:
# first utilize a convolution definition from d2l
def corr2d(X, K):
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [ ]:
import torch

4.1

In [ ]:
def avg_pool(X,ker_size):
  kernel = torch.full((ker_size,ker_size),(1/ker_size**2))
  return corr2d(X,kernel)

In [ ]:
X = torch.randint(1,100,(7,7))
avgX = avg_pool(X,4)
print(X)
print(avgX)

NameError: name 'corr2d' is not defined

4.2.1

In [ ]:
# max pooling

# first take an a and b
a = torch.tensor([-100])
b = torch.tensor([-255])

In [ ]:
aMb = a - b
bMa = -aMb
aPb = a + b
MaPb = -aPb


In [ ]:
# if b > a, aMb is - and bMa is +, otherwise bMa is - and aMb is +
# Thus if we ReLU both, one will return a 0 and the other will give the difference, we then add these together to get the absolute difference

dif = torch.relu(aMb) + torch.relu(bMa)
add = torch.relu(aPb) - torch.relu(MaPb)

# now we have the absolute difference of the two numbers, if we add the product of these numbers we get as a resultant twice the amount of the larger number (assuming both are positive)
# assume a - b + a + b = 2*a

max = (dif + add)/2
print(max)

tensor([-100.])


4.2.2

In [ ]:
# therefore an exact max function would be done using the following function
import torch
def my_max(a,b):
  ta = torch.tensor([a])
  tb = torch.tensor([b])
  return (torch.relu(ta-tb)+torch.relu(tb-ta)+ta+tb)/2

In [ ]:
my_max(-100,-10)

tensor([-10.])

In [ ]:
# assuming no stride and an n x n grid
# 2 x 2 comparisons would use a convolution of torch.tensor([[0,1],[-1,0]]) and torch.tensor([[0,-1],[1,0]]) and , which would be ReLU-ed then summed with
# convolution of torch.tensor([[0,1],[1,0]]) well as a
# torch.tensor([[1,0],[0,-1]]) and torch.tensor([[-1,0],[0,1]]), which would be ReLU-ed and summed, with a convolution of torch.tensor([[0,1],[1,0]]).
# These should somehow be compared with each other.

# to do this we combine the two previous resultants into either one tensor, alternating each value in each row like ([[1t11,2t11,1t12,2t12,...],[1t21,2t21,1t22,2t22,...],...])
# then pass two convolutions of stride 2 and torch.tensor([1,-1]) and torch.tensor([-1,1]) which would then be ReLU-ed and summed along with an original convolution of
# stride 2, torch.tensor([1,1])

# similarly perhaps, we could do a 1x1 convolution with the two seperate solutions as channels and merely use the convolutions of [-1],[1] and [1],[-1] then ReLU-ed and summed,
# with an addition of a [1],[1] convolution Though this would add convusion in a space of many many channels, thus use the original idea

def max_2x2(in_var):
  con11 = torch.relu(corr2d(in_var,torch.tensor([[0,1],[-1,0]])))
  con12 = torch.relu(corr2d(in_var,torch.tensor([[0,-1],[1,0]])))
  con13 = torch.relu(corr2d(in_var,torch.tensor([[0,1],[1,0]])))
  con14 = torch.relu(corr2d(in_var,torch.tensor([[0,-1],[-1,0]])))
  con21 = torch.relu(corr2d(in_var,torch.tensor([[1,0],[0,-1]])))
  con22 = torch.relu(corr2d(in_var,torch.tensor([[-1,0],[0,1]])))
  con23 = torch.relu(corr2d(in_var,torch.tensor([[1,0],[0,1]])))
  con24 = torch.relu(corr2d(in_var,torch.tensor([[-1,0],[0,-1]])))
  con1 = (con11 + con12 + con13 - con14)/2
  con2 = (con21 + con22 + con23 - con24)/2
  return (torch.relu(con1 - con2) + torch.relu(con2 - con1) + torch.relu(con1 + con2) - torch.relu(-con1 - con2))/2
def max_3x3(in_var):
  return(max_2x2(max_2x2(in_var)))

X = torch.tensor([[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15],[16,17,18,19,20],[21,22,23,24,25]])
max = max_2x2(X)
max = max_3x3(max)
print(max)

tensor([[19., 20.],
        [24., 25.]])


4.2.3

A 2x2 convolution/ReLU max pooling requires four layers, one to make two seperate replicas of the initial with 2 x 1 channels, one of 4 x 2 channels and one of 2 x 4 channels, and then one of 1 x 2 channels.

A 3x3 max pooling would just need for two of these to be done. ie 8 layers.

4.3

For c chanels, (h,w) input size, (p_h,p_w) padding, (p_h,p_w) pooling kernel size and (s_h,s_w) strides we can calculate the outcome's shape.

The output size will be floor((h+p_h-p_h+1)/s_h,(w+p_w-p_w+1)/s_w) or floor(h+1/s_h,w+1/s_w). This results in a total of floor(h+1/s_h)*floor(w+1/s_w) total pooling instances, which means the entire computational cost is this number times the computational cost for one case of pooling.